In [1]:
import re
import urllib.request

In [2]:
with open("verdict.txt","r", encoding='utf-8') as f:
    data = f.read()
    print(data[:99])
    print("The file is closed: ", f.closed)

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
The file is closed:  False


In [3]:
len(data)

20479

In [4]:
li = [2, 3, 4, 5, 6, 7, 8, 9, 10]
app = [ x**2 for x in li if x % 2]
app

[9, 25, 49, 81]

In [5]:
preprocessed_data =  re.split(r'([,.:;?_!"()\']|--|\s)', data)
preprocessed_data = [item.strip() for item in preprocessed_data if item.strip()]
print(len(preprocessed_data))

4690


In [6]:
print(preprocessed_data[:20])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was']


In [7]:
all_words = sorted(set(preprocessed_data))
vocab_size= len(all_words)
vocab_size

1130

In [8]:
vocab = { token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [9]:
class SimpleTokenizerV1:

    def __init__(self, vocab):
        self.str_to_int = vocab 
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [ item.strip() for item in preprocessed if item.strip()]

        ids = [self.str_to_int[s] for s in preprocessed]

        return ids

    def decode(self, ids):

        text = " ".join([self.int_to_str[i] for i in ids])

        text  = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text        

In [10]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted you know, " Mrs . Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [11]:
print(tokenizer.decode(ids))

" It' s the last he painted you know," Mrs. Gisburn said with pardonable pride.


In [12]:
all_tokens = sorted(list(set(preprocessed_data)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = { token: integer for integer, token in enumerate(all_tokens) }

len(vocab)

1132

In [13]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [14]:
class SimpleTokenizerV1:

    def __init__(self, vocab):
        self.str_to_int = vocab 
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [ item.strip() for item in preprocessed if item.strip()]
        preprocessed = [ item if item in self.str_to_int 
                            else '<|unk|>' for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]

        return ids

    def decode(self, ids):

        text = " ".join([self.int_to_str[i] for i in ids])

        text  = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)

        return text       

In [15]:
text1 = "Hello, do you like tea?"
text2 = "no that much but i drink often"
text = " <|endoftext|> ".join([text1, text2])
text

'Hello, do you like tea? <|endoftext|> no that much but i drink often'

In [16]:
tokenizer = SimpleTokenizerV1(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 709, 987, 691, 239, 1131, 1131, 724]


In [17]:
from importlib.metadata import version
import tiktoken 
print(version('tiktoken'))

0.8.0


In [18]:
text = "Hello do you like chai it is dhoodh plus pani kwirkdskd"
tokenizer = tiktoken.get_encoding("gpt2")

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>" })
print(integers)

[15496, 466, 345, 588, 442, 1872, 340, 318, 288, 2894, 71, 5556, 279, 3216, 479, 86, 14232, 67, 8135, 67]


In [19]:
strings = tokenizer.decode(integers)
print(strings)

Hello do you like chai it is dhoodh plus pani kwirkdskd


In [20]:
data[:99]
print(len(data))

20479


In [21]:
enc_text = tokenizer.encode(data)
print(len(enc_text))

5145


In [22]:
import torch
from torch.utils.data import Dataset, DataLoader 


In [23]:
class GPTDatasetV1(Dataset):
    def __init__(self,txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)


        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]




In [24]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                        stride=128, shuffle=True,drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length,stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [25]:
dataloader = create_dataloader_v1(
    data, batch_size=1, max_length=4, stride=1, shuffle=False
)
data_iter = iter(dataloader)
first_batch  = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [26]:
dataloader = create_dataloader_v1(
    data, batch_size=8, max_length=4, stride=4,shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n",targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [27]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(6,3)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
